<a href="https://colab.research.google.com/github/x-legion/legions-sd-notebooks/blob/main/legions_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Install Dependencies

import os
from IPython.display import clear_output

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2
!pip install -q xformers==0.0.20 triton==2.0.0 -U
!pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf

repo_dir = "/content/ComfyUI"

!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd {repo_dir}
!pip install -q -r requirements.txt

%cd {repo_dir}/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/WASasquatch/was-node-suite-comfyui.git
!git clone https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes.git
!git clone https://github.com/EllangoK/ComfyUI-post-processing-nodes.git
!git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale --recursive
!git clone https://github.com/BlenderNeko/ComfyUI_TiledKSampler.git

clear_output()
print("\033[34m>> Finished installing Dependencies\033[0m")

In [ ]:
# @title 2. Download Models

import os
import re
import csv
from IPython.display import clear_output
!apt install aria2 -y

%cd {repo_dir}

# Set up model directories in Colab root folder
model_dir = "models/upscale_models" # @param ["models/checkpoints", "models/vae", "models/loras", "models/embeddings", "models/controlnet", "models/upscale_models"] {allow-input: true}
download_links = "https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/lollypop.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/DPED.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_foolhardy_Remacri.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_Nickelback_70000G.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_NMKD-YandereNeoXL_200k.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_NMKD-Superscale-SP_178000_G.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x-UltraSharp.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x-AnimeSharp.pth" # @param ["https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors,https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors", "https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors", "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", "https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/lollypop.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/DPED.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_foolhardy_Remacri.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_Nickelback_70000G.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_NMKD-YandereNeoXL_200k.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x_NMKD-Superscale-SP_178000_G.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x-UltraSharp.pth,https://huggingface.co/Linaqruf/stolen/resolve/main/upscaler/4x-AnimeSharp.pth"] {allow-input: true}

# Parse links
links = list(csv.reader([download_links]))[0]

for link in links:
  filename = os.path.basename(link)
  print(f"Downloading {filename} to {model_dir}")
  !aria2c -x 16 -s 16 -d {model_dir} -o {filename} {link}
  clear_output()

clear_output()
print("\033[31m>> checkpoints\033[0m")
!ls models/checkpoints

print("\033[34m>> vae\033[0m")
!ls models/vae

print("\033[33m>> loras\033[0m")
!ls models/loras

print("\033[32m>> controlnet\033[0m")
!ls models/controlnet

print("\033[35m>> upscale_models\033[0m")
!ls models/upscale_models

In [ ]:
# @title 3. Run ComfyUI

import os
from IPython.display import clear_output
from google.colab import drive


launch_arguments = '' # @param {type:"string"}
drive_output = True # @param {type:"boolean"}
tunnel = "Cloudflared" # @param ["Cloudflared", "Localtunnel", "Iframe"]

if drive_output:
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive
    !mkdir comfyui_output
    launch_arguments += " --output-directory /content/drive/MyDrive/comfyui_output"

%cd {repo_dir}

if tunnel == "Cloudflared":

  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
  import atexit, requests, subprocess, time, re
  from random import randint
  from threading import Timer
  from queue import Queue
  def cloudflared(port, metrics_port, output_queue):
      atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
      attempts, tunnel_url = 0, None
      while attempts < 10 and not tunnel_url:
          attempts += 1
          time.sleep(3)
          try:
              tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
          except:
              pass
      if not tunnel_url:
          raise Exception("Can't connect to Cloudflare Edge")
      output_queue.put(tunnel_url)
  output_queue, metrics_port = Queue(), randint(8100, 9000)
  thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
  thread.start()
  thread.join()
  tunnel_url = output_queue.get()
  os.environ['webui_url'] = tunnel_url
  clear_output()
  print(">> link:")
  print(tunnel_url)
  print("\n\n")

  !python main.py --dont-print-server {launch_arguments}

elif tunnel == "Localtunnel":

  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")
    clear_output()
    print("\n\n")
    print(">> The password/enpoint ip for localtunnel is:\n", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    print("\n\n")
    print(">> link:")
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py --dont-print-server {launch_arguments}

elif tunnel == "Iframe":
  import threading
  import time
  import socket
  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    from google.colab import output
    output.serve_kernel_port_as_iframe(port, height=1024)
    print("to open it in a window you can open this link here:")
    output.serve_kernel_port_as_window(port)

  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py --dont-print-server {launch_arguments}

In [ ]:
# @title Show launch_arguments

!python main.py --help
